## Avocado Price Prediction
____

## Project Goal
Beginner project using avocado prices dataset to build simple price prediction models.

## Data Source
The dataset can be found on kaggle open datasets, or via link [here](https://www.kaggle.com/neuromusic/avocado-prices). Additional dataset information can be found [here](https://cran.r-project.org/web/packages/avocado/vignettes/a_intro.html).

## Environment Setup

In [ ]:
# data manipulation and visulization
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

np.warnings.filterwarnings('ignore')

# preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.preprocessing import OneHotEncoder

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# making a function for examining data
def data_research(data, data_name='data'):
    # basic
    print(f'Examining "{data_name}"')
    display(data.head())
    display(data.tail())
    display(data.info())
    display(data.describe( include='all'))
    display(data.columns)
    
    # duplicates
    duplicates = data.duplicated().sum()
    if duplicates > 0:
        print('There are no duplicated entries.')
    else:
        print(f'There are {duplicates} duplicates.')
        
    # missing
    data_missing = pd.DataFrame(round(data.isnull().sum() / data.shape[0] * 100, 2))
    if data_missing[0].sum() > 0:
        print(f'Missing values in the "{data_name}":')
        data_missing.plot(kind='bar')
    else:
        print(f'There are no missing values in "{data_name}".')
    
    # unique values
    for i in data.columns:
        if data[i].dtype == 'object' or data[i].dtype == 'str':
            print(data[i].unique())

In [ ]:
# loading dataset
df = pd.read_csv("../input/avocado-prices/avocado.csv")

## Check the dataset

In [ ]:
data_research(df, data_name='Avocado Prices')

In [ ]:
print(df['type'].value_counts())
sns.countplot('type', data=df, palette='Set3')
plt.show()

In [ ]:
print(df['region'].value_counts())
print('\n', 'There are:', len(df['region'].unique()), 'unique values in the feature')

Comments on initial dataset features:
1. unnamed - useless column, will drop.
1. Date - year is already a feature field, extract month, week, perhaps day for further analysis
1. AveragePrice - per unit price (that is per avocado)
1. Total Volume - total number of avocado sold
1. 4046, 4225, 4770 - PLUs (product lookup code) of Hass Avocados, other varities are not listed, according to the [Hass Avocado Board](https://loveonetoday.com/how-to/identify-hass-avocados/), PLU4046 is small/medium Hass, PLU4225 is large Hass, PLU4770 is extra large Hass. Make additional calulated field for non-Hass avocados by subtracting the 3 types of Hass from total volume.
1. Total Bags - total number of bags sold (=Small + Large + XLarge bags)
1. type - conventional and organic, ~50% each
1. region - where the observation is recorded, 54 unique regions, need cleaning, contains TotalUS, states, cities, and other regions that are not precise, some regions are subsets of other regions

## Data Cleaning & Wrangling

Dropping column and changing column names to all lower cases connected with underline instead of spaces.

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df = df.rename(columns={"Date": "date","AveragePrice": "average_price","Region":"region","Total Volume":"total_volume","4046":"small_hass","4225":"large_hass","4770":"xlarge_hass","Total Bags":"total_bags","Small Bags":"small_bags","Large Bags":"large_bags","XLarge Bags":"xlarge_bags"})
df.columns

Additional calulated field for non-Hass avocados

In [ ]:
df['non_hass'] = df['total_volume']-df['small_hass']-df['large_hass']-df['xlarge_hass']

Change 'date' to datetime from object

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['date'].dtypes

Extract month from date field as an additional field

In [ ]:
df['month'] = df['date'].dt.month
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
# monday = 0
df['day_of_week'] = df['date'].dt.dayofweek

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20,10))

sns.countplot('year', data=df, ax=ax[0,0], palette='BuGn_r')
sns.countplot('month', data=df, ax=ax[0,1], palette='BuGn_r')
sns.countplot('day', data=df, ax=ax[1,0], palette='BuGn_r')
sns.countplot('day_of_week', data=df, ax=ax[1,1], palette='BuGn')

plt.show()

Comments on date features:
* 2015, 2016, 2017 have similar amount of data, 2018 has less due to the fact that the dataset is only up to the of March of 2018.
* Same reason as above, the first 3 months have more records.
* Due to the fact that data is always recorded on day 6 (Sunday), therefore, the day_of_week feature is redundant and can be dropped.

In [ ]:
df.drop('day_of_week', axis=1, inplace=True)

Change 'type' and 'region' to string

In [ ]:
df = df.convert_dtypes()
df.dtypes

Sort dataframe by date in ascending order

In [ ]:
df.sort_values(by=['date'], inplace=True)
df.head()

Only keeping regions that are not overall regions, in other words, only keeping the regions at the lowest level. This is to avoid potential multicollinerarity.

In [ ]:
regionsToRemove = ['California', 'West', 'Plains', 'SouthCentral', 'Southeast', 'Midsouth', 'GreatLakes', 'Northeast', 'TotalUS']
df = df[~df.region.isin(regionsToRemove)]
len(df.region.unique())

## EDA

### Correlations

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), vmax=1.0, vmin=-1.0, annot=True)

In [ ]:
columns_for_research = ['average_price', 'total_volume', 'small_hass','large_hass','xlarge_hass', 'non_hass', 'total_bags', 'small_bags', 'large_bags', 'xlarge_bags', 'type']
g = sns.pairplot(data=df[columns_for_research], hue="type")
g.fig.suptitle("Distributions and scatter plots for each variable depending on type", y=1.01, size=16)
for ax in g.axes.flat: 
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

Comments:

* inverse correlation between volume related columns and ave_price
* conventional avocado prices seem to be smaller in general in comparison to organic avocados
* discover 'non-hass' is exactly the same as 'total_bag', further investigation in dataset information [here](https://cran.r-project.org/web/packages/avocado/vignettes/a_intro.html), realize that PLU columns are bulk sell, and rest are sold in bags, where the bags could contain different sizes of avocados. Drop the non-hass column before model building since duplication.
* Also before model building, drop total volumn and total bags to avoid multicollinearity, since total volume = small hass + large hass + xlarge hass + total bags and total bags = small bags + large Bags + xlarge bags 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14,5))

sns.barplot(x='type', y='average_price', data=df, palette='Set3', ax=ax[0])
sns.barplot(x='type', y='total_volume', data=df, palette='Set3', ax=ax[1], estimator=sum, ci=None)
plt.show()

display(df.groupby('type')['average_price'].mean())
display(df.groupby('type')['total_volume'].sum())

Comment:
* confirm conventional avocados tends to be cheaper but sell a lot more in comparison to organic avocados

### Dates

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(23,10))

df['year_month'] = df['date'].dt.to_period('M')
grouped = df.groupby('year_month')[['average_price', 'total_volume']].mean()

ax[0].plot(grouped.index.astype(str), grouped['average_price'].tolist())
ax[0].tick_params(labelrotation=90)
ax[0].set_ylabel('average_price')

ax[1].plot(grouped.index.astype(str), grouped['total_volume'].tolist())
ax[1].tick_params(labelrotation=90)
ax[1].set_ylabel('total_volume')

plt.show()

Comments:
* confirm inverse corelation between average price and total volume
* there seem to be seasonality, lower volumes during fall and winter, higher during spring and summer

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))

df['quarter'] = df['date'].dt.quarter

sns.barplot(x='quarter', y='total_volume', data=df, palette='Greens_r', ci=None, ax=ax[0])
sns.barplot(x='quarter', y='average_price', data=df, palette='Greens_r', ci=None, ax=ax[1])

plt.show()

quarter = df.groupby('quarter')[['total_volume', 'average_price']].mean()
display(quarter)

Comment:
* there is a pattern of light seasonality

In [ ]:
df.shape

## Regression Models for Price Prediction

### Train and Test Split

Since the data is a time series data (gives weekly avocado prices between Jan 2015 and Mar 2018).
I sorted it by Date and then split it due to date manually (not randomly), to preserve the 'times series effect' on it.
I determined to use 2015, 2016, 2017 as the train set, and use the first three months in 2018 as the test set.

In [ ]:
df_model = df[['average_price', 'date', 'total_volume', 'month', 'type', 'region']]
#df_model = df.drop(['year_month', 'non_hass', 'total_volume', 'total_bags', 'quarter'], axis=1)

test = df_model[df_model['date'] > '2018-01-01']
train = df_model[df_model['date'] < '2018-01-01']

test = test.drop(['date'], axis=1)
train = train.drop(['date'], axis=1)

test_target = test['average_price']
train_target = train['average_price']
test_features = test.drop(['average_price'], axis=1)
train_features = train.drop(['average_price'], axis=1)

### Encoding Categorical Features

In [ ]:
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)

encoder.fit(train_features[['type', 'region']]) 

encoded_train = pd.DataFrame(encoder.fit_transform(train_features[['type', 'region']]))
encoded_test = pd.DataFrame(encoder.transform(test_features[['type', 'region']]))

encoded_train.index = train_features.index
encoded_test.index = test_features.index 

cattraincol = train_features.drop(['type', 'region'], axis=1)
cattestcol = test_features.drop(['type', 'region'], axis=1)

train_features = pd.concat([cattraincol, encoded_train], axis=1)
test_features = pd.concat([cattestcol, encoded_test], axis=1)

### Decision Tree and Random Forest

In [ ]:
tree = DecisionTreeRegressor(max_depth=2, random_state=0).fit(train_features, train_target)
preds = tree.predict(test_features)
RMSE_tree = mean_squared_error(test_target, preds, squared=False)
print("\n", "tree RMSE is:", RMSE_tree)

In [ ]:
RForest = RandomForestRegressor().fit(train_features, train_target)
preds = RForest.predict(test_features)
RMSE_RForest = mean_squared_error(test_target, preds, squared=False)
print("\n", "RForest RMSE is:", RMSE_RForest)

### Linear Models

In [ ]:
LinReg = LinearRegression().fit(train_features, train_target)
preds = LinReg.predict(test_features)
RMSE_LinReg = mean_squared_error(test_target, preds, squared=False)
print("\n", "LinReg RMSE is:", RMSE_LinReg)

In [ ]:
Knn = KNeighborsRegressor().fit(train_features, train_target)
preds = Knn.predict(test_features)
RMSE_Knn = mean_squared_error(test_target, preds, squared=False)
print("\n", "Knn RMSE is:", RMSE_Knn)

In [ ]:
ridge = Ridge().fit(train_features, train_target)
preds = ridge.predict(test_features)
RMSE_Ridge = mean_squared_error(test_target, preds, squared=False)
print("\n", "Ridge RMSE is:", RMSE_Ridge)

In [ ]:
lasso = Lasso().fit(train_features, train_target)
preds = lasso.predict(test_features)
RMSE_lasso = mean_squared_error(test_target, preds, squared=False)
print("\n", "Lasso RMSE is:", RMSE_lasso)

In [ ]:
print("\n", "tree RMSE is:", RMSE_tree)
print("\n", "RForest RMSE is:", RMSE_RForest)
print("\n", "LinReg RMSE is:", RMSE_LinReg)
print("\n", "Knn RMSE is:", RMSE_Knn)
print("\n", "Ridge RMSE is:", RMSE_Ridge)
print("\n", "Lasso RMSE is:", RMSE_lasso)

Traditional regression models performs relatively closely. Choose the RMSE value cloest to zero to display.

In [ ]:
ridge = Ridge().fit(train_features, train_target)
preds = ridge.predict(test_features)
RMSE_Ridge = mean_squared_error(test_target, preds, squared=False)
print("\n", "Ridge RMSE is:", RMSE_Ridge)

In [ ]:
import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

groupBy_whole = df_model.groupby('date').mean()

new_frame = groupBy_whole[groupBy_whole.index > '2018-01-01']
new_frame = new_frame['average_price']
new_frame['Predictions'] = preds

scatter1 = go.Scatter(x=groupBy_whole.index, y=groupBy_whole['average_price'], name="Actual")
scatter2 = go.Scatter(x=new_frame.index, y=new_frame['Predictions'], name="Predictions")

data = [scatter1, scatter2]
layout=go.Layout(title="Prediction vs Actual Test Points", xaxis={'title':'Date'}, yaxis={'title':'Average Price'})
figure=go.Figure(data=data,layout=layout)
iplot(figure)

## Time Series Model with FB Prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
groupBy_whole.reset_index(inplace=True)

In [ ]:
df_model_prophet = groupBy_whole[['date', 'average_price']]
test_prophet = df_model_prophet[df_model_prophet['date'] > '2018-01-01']
train_prophet = df_model_prophet[df_model_prophet['date'] < '2018-01-01']

train_prophet = train_prophet.rename(columns={'date':'ds', 'average_price':'y'})
test_prophet  = test_prophet.rename(columns={'date':'ds', 'average_price':'y'})
display(df_model_prophet.plot(x='date', y='average_price', kind="line"))

In [ ]:
train_prophet["y"] =[float(v) for v in train_prophet.y] #fixing data type Value Error of 'y' when fitting

In [ ]:
prop = Prophet()
prop.fit(train_prophet)

future = prop.make_future_dataframe(periods = 12, freq = 'w')
y_hat = prop.predict(future)

fig1 = prop.plot(y_hat)

df_res = pd.merge(test_prophet, y_hat[["ds","yhat"]], on = "ds", how = "left")
df_res["difference"] = abs(df_res["yhat"] - df_res["y"])
rmse = mean_squared_error(df_res.y, df_res.yhat, squared=False)
rmse

In [ ]:
new_frame2 = df_model_prophet
new_frame2['Predictions'] = y_hat.yhat

scatter1 = go.Scatter(x=df_model_prophet['date'], y=df_model_prophet['average_price'], name="Actual")
scatter2 = go.Scatter(x=new_frame2['date'], y=new_frame2['Predictions'], name="Predictions")

data = [scatter1, scatter2]
layout=go.Layout(title="Prediction vs Actual Test Points", xaxis={'title':'Date'}, yaxis={'title':'Average Price'})
figure=go.Figure(data=data,layout=layout)
iplot(figure)

FB prophet seems to perform better than the traditional models on our dataset.

## References

Appreciations given to the following notebooks from other Kaggler. Some codes, logic or thoughts where taken directly from them.
* [Avocado | EDA](https://www.kaggle.com/kslarwtf/avocado-eda) by KS_LAR_WTF.
* [EDA + Lasso](https://www.kaggle.com/nilanml/eda-lasso) by NILAN.
* [Regression Models to Predict Average Avocado Price](https://www.kaggle.com/tahirbey/regression-models-to-predict-average-avocado-price/notebook) by TAHIRBEY.
* [Statistical Avo: EDA, Analysis and ML](https://www.kaggle.com/jaimebecerraguerrero/statistical-avo-eda-analysis-and-ml) by JAIME BECERRA GUERRERO.
* [Avocado Price Prediction](https://www.kaggle.com/mohaiminul101/avocado-price-prediction) by MOHAIMINUL ISLAM.
* [LinReg,KNN,SVR,DecisionTreeRandomForest,TimeSeries](https://www.kaggle.com/ladylittlebee/linreg-knn-svr-decisiontreerandomforest-timeseries) by MCS.